Creating Spark Session and configuring CPU and Memory resources for this Spark Session. 
Read all the downloaded Parquet files in a Dataframe (df) from "parquet_files" folder. 
Infer schema from parquet files automatically and create a Table like temporary view "ccnews".
Now we can query data from "ccnews" just like any other tabe using SQL queries.

In [158]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master('local[4]') \
    .appName('advertisement') \
    .config('spark.executor.memory', '10gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

df = spark.read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .option("compression", "snappy") \
    .parquet("./parquet_files/")

df.createOrReplaceTempView("ccnews")

Following query will list down the most trending article of every month in year 2023 with English language
We have partitioned the data by Year and Month and select the article with maximum page visits per month
Query result is organized per month for the year mentioned in HAVING clause at the end of 3rd last line (published_year=2016)

In [165]:
spark.sql("select categories as trending_article, frequency as page_visits, published_month, published_year \
from (select *, ROW_NUMBER() OVER (PARTITION BY published_year, published_month ORDER BY frequency DESC) as rank \
from (select categories, count(1) as frequency, YEAR(published_date) as published_year, \
MONTH(published_date) as published_month from ccnews \
where language='en' GROUP BY categories, published_year, published_month \
HAVING LENGTH(categories)!=0 and categories!='none' and published_year=2023 \
ORDER BY published_year desc, published_month asc, frequency desc)) \
where rank <= 1 ORDER BY published_month ASC ").show()

+----------------+-----------+---------------+--------------+
|trending_article|page_visits|published_month|published_year|
+----------------+-----------+---------------+--------------+
|        National|        140|              1|          2023|
|Northern Ireland|         35|              2|          2023|
|   International|          5|              3|          2023|
|        National|        150|              4|          2023|
|   Entertainment|         17|              5|          2023|
|          Sports|         24|              6|          2023|
|            News|      25777|              7|          2023|
|     Nova Scotia|          3|              8|          2023|
|         Toronto|          2|              9|          2023|
|   International|        263|             10|          2023|
|        Olympics|          2|             11|          2023|
|  War in Ukraine|         61|             12|          2023|
+----------------+-----------+---------------+--------------+



Following query will list down the most trending article of every month in year 2022 with English language.
You can visualize data for another year by changing value of "published_year" to the year of your choice.

In [167]:
spark.sql("select categories as trending_article, frequency as page_visits, published_month, published_year \
from (select *, ROW_NUMBER() OVER (PARTITION BY published_year, published_month ORDER BY frequency DESC) as rank \
from (select categories, count(1) as frequency, YEAR(published_date) as published_year, \
MONTH(published_date) as published_month from ccnews \
where language='en' GROUP BY categories, published_year, published_month \
having LENGTH(categories)!=0 and categories!='none' and published_year=2022 \
ORDER BY published_year desc, published_month asc, frequency desc)) \
where rank <= 1 ORDER BY published_month ASC ").show()

+----------------+-----------+---------------+--------------+
|trending_article|page_visits|published_month|published_year|
+----------------+-----------+---------------+--------------+
|        Pakistan|         13|              1|          2022|
|            News|         24|              2|          2022|
|  War in Ukraine|          8|              3|          2022|
|          hybrid|          2|              4|          2022|
|            news|         15|              5|          2022|
|        Politics|         43|              6|          2022|
|  War in Ukraine|         15|              7|          2022|
|   Entertainment|          2|              8|          2022|
|        National|        149|              9|          2022|
| Latest;Pakistan|         71|             10|          2022|
|           Local|         62|             11|          2022|
|            News|      21428|             12|          2022|
+----------------+-----------+---------------+--------------+

